In [ ]:
from classify import data_pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce

# Rzepa

In [ ]:
df = data_pipeline("rzepa")
df.drop(columns=['vectorized'], inplace=True)

In [ ]:
df['Rank'] = None

## Filter I

In [ ]:
def filter_by_conditions(df, condition_list):
    selection_mask = reduce(lambda x, y: x & y, condition_list)
    df.loc[selection_mask, 'Rank'] = 'Test'
    return df[selection_mask]

In [ ]:
condition_list = []
condition_list.append(df['ngram_sum_squared_to_total'] > 1)
condition_list.append(df['proba'] > 0.9)
condition_list.append(df['weak_count'] < 0.9)
condition_list.append(df['words'].apply(len) > 2)
condition_list.append(df['Rank'].isna())

df_selected = filter_by_conditions(df, condition_list)

In [ ]:
df_sample = df_selected.sample(100)
df_sample.to_csv("data.csv")

In [ ]:
# Approved
df.loc[df['Rank'] == 'Test', 'Rank'] = 'Filter1'

## Filter II

In [ ]:
df_left = df[df['Rank'].isna()]
condition_list = [df_left['weak_count']  < 0.3,
                  df_left['ngram_sum_squared_to_total'] > 1]
df_selected = filter_by_conditions(df_left, condition_list)

In [ ]:
sns.jointplot(df_selected, x = 'weak_count', y = 'ngram_sum_squared')
plt.yscale('log')

In [ ]:
df_selected.sample(100).to_csv('data.csv')